In [28]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# 병목 구조
def bottleneck_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)

    x = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    x = layers.Conv2D(filters * 4, kernel_size=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)

    if downsample or shortcut.shape[-1] != filters * 4:
        shortcut = layers.Conv2D(filters * 4, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x


# ResNet-50
def create_resnet50_model(input_shape):
    input_layer = Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    filter_sizes = [64, 128, 256, 512]
    num_blocks = [3, 4, 6, 3]  # ResNet-50에서 각 블록별 반복 횟수
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = bottleneck_block(x, filters, downsample=(i == 0 and filters != 64))
    
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# MLP
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation = mish)(input_layer)
    x = layers.Dense(64, activation = mish)(x)
    x = layers.Dense(3, activation = mish)(x) # 나중에 3 -> 2로 수정 예정. (3으로 인한 특징 추출{가설}을 파악하기 위함)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs= x)

folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img\hsh"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results\hsh.csv"

csv_data = pd.read_csv(csv_path, header=0, encoding='utf-8')

# 이미지 파일 읽기 및 session/point 추출
image_files = glob.glob(os.path.join(folder_path, '*.jpg'))
image_data = []
for file in image_files:
    session, point = extract_session_and_point(os.path.basename(file))
    if session is not None and point is not None:
        image_data.append({'Filename': file, 'Session': session, 'Point': point})
image_df = pd.DataFrame(image_data)

merged_data = pd.merge(image_df, csv_data, left_on=['Session', 'Point'], right_on=['session', 'point'], how='inner')

valid_labels = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45]
filtered_data = merged_data[merged_data['Point'].isin(valid_labels)]

right_images = filtered_data[filtered_data['Filename'].str.contains('right', case=False)]
left_images = filtered_data[filtered_data['Filename'].str.contains('left', case=False)]

right_images_array = np.array([load_and_preprocess_image(path) for path in right_images['Filename']])
left_images_array = np.array([load_and_preprocess_image(path) for path in left_images['Filename']])

features = filtered_data.drop(['Filename', 'session', 'point', 'Session', 'Point'], axis=1).values
labels = filtered_data['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
labels = to_categorical(labels, num_classes=len(valid_labels))

right_eye_model = create_resnet50_model((128, 128, 1))
left_eye_model = create_resnet50_model((128, 128, 1))
mlp_model = create_mlp_model(features.shape[1:])

combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])

x = layers.Dense(256, activation=mish)(combined_input)
x = layers.Dropout(0.5)(x)
output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)

combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = combined_model.fit([right_images_array, left_images_array, features], labels, 
                             epochs=50, batch_size=1, validation_split=0.2)

train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Train Accuracy: {train_accuracy}, Validation Accuracy: {val_accuracy}")

model_save_folder = r"C:\Users\admin\Desktop\sihoon\webcam\result\1124"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)
combined_model.save(os.path.join(model_save_folder, "combined_model.h5"))

predictions = combined_model.predict([right_images_array, left_images_array, features])
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(labels, axis=1)

incorrect_predictions = np.where(actual_classes != predicted_classes)[0]

data = {
    'Right Image': [os.path.basename(right_images[idx]) for idx in range(len(right_images))],
    'Left Image': [os.path.basename(left_images[idx]) for idx in range(len(left_images))],
    'Actual Class': actual_classes.tolist(),
    'Predicted Class': predicted_classes.tolist(),
    'Features': [features_array[idx].tolist() for idx in range(len(features_array))]
}
df = pd.DataFrame(data)

csv_save_folder = r"C:\Users\admin\Desktop\sihoon\webcam\result\1124"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)
df.to_csv(os.path.join(csv_save_folder, "predictions.csv"), index=False)

# 잘못 예측한 데이터만 저장
data = {
    'Right Image': [os.path.basename(right_images[idx]) for idx in incorrect_predictions],
    'Left Image': [os.path.basename(left_images[idx]) for idx in incorrect_predictions],
    'Actual Class': actual_classes[incorrect_predictions],
    'Predicted Class': predicted_classes[incorrect_predictions],
    'Features': [features_array[idx].tolist() for idx in incorrect_predictions]
}
df_incorrect = pd.DataFrame(data)

# 잘못 예측한 데이터를 저장할 CSV 경로
incorrect_csv_save_folder = r"C:\Users\admin\Desktop\sihoon\webcam\result\1124"
if not os.path.exists(incorrect_csv_save_folder):
    os.makedirs(incorrect_csv_save_folder)

incorrect_csv_path = os.path.join(incorrect_csv_save_folder, "incorrect_predictions.csv")
df_incorrect.to_csv(incorrect_csv_path, index=False)

Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_1744', 'keras_tensor_1914', 'keras_tensor_2084']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


460/460 ━━━━━━━━━━━━━━━━━━━━ 204s 350ms/step - accuracy: 0.0556 - loss: 7.4956 - val_accuracy: 0.0435 - val_loss: 3.5182
Epoch 2/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 160s 348ms/step - accuracy: 0.0383 - loss: 3.2104 - val_accuracy: 0.0261 - val_loss: 3.3418
Epoch 3/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 350ms/step - accuracy: 0.0590 - loss: 3.1922 - val_accuracy: 0.0435 - val_loss: 3.1355
Epoch 4/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 349ms/step - accuracy: 0.0536 - loss: 3.1382 - val_accuracy: 0.0435 - val_loss: 3.1401
Epoch 5/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 350ms/step - accuracy: 0.0471 - loss: 3.1396 - val_accuracy: 0.0000e+00 - val_loss: 3.1343
Epoch 6/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 350ms/step - accuracy: 0.0405 - loss: 3.1957 - val_accuracy: 0.0435 - val_loss: 3.1355
Epoch 7/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 350ms/step - accuracy: 0.0501 - loss: 3.1596 - val_accuracy: 0.0522 - val_loss: 3.1277
Epoch 8/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 161s 350ms/step - accuracy: 0.0260 - loss: 

Train Accuracy: 0.021739130839705467, Validation Accuracy: 0.043478261679410934


NameError: name 'features_array' is not defined